<a href="https://colab.research.google.com/github/taekyungss/Hands_on_ML_Study/blob/main/Chapter3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Chapter3. 분류
### classfication

In [ ]:
# 필요한 라이브러리 불러오기

import numpy as np

지도 학습에서 가장 대표적인 작업이 회귀와 분류
(회귀): 값을 예측하는 것
(분류): 클래스 예측

In [ ]:
# MINIST 데이터셋 내려받는 코드

from sklearn.datasets import fetch_openml

mnist = fetch_openml("mnist_784", version=1, as_frame = False)
mnist.keys()

받아와진 데이터셋 안의 keys에는 
데이터셋을 설명하는 DESCR, 샘플이 한의 행, 특성이 하나의 열로 구성된 data키,

레이블 배열을 담은 target 키 등이 존재함.

In [ ]:
print(mnist["data"])
print("\n====\n")
print(mnist["target"])

X에는 데이터를 담고, y에는 레이블이 담겨 있는 것.

In [ ]:
X,y = mnist["data"], mnist["target"]
print(X.shape)
print(y.shape)

이미지가 70000개가 존재, 특성은 784개 존재

이유: 이미지가 28*28 픽셀이므로?

--> 그다음 해야할 작업 데이터셋에서 이미지 하나를 확인해보자

방법: 샘플의 특성 벡터를 추출해서 28*28배열로 크기를 변환시킨후, 맷플롯립의 imshow()함수를 사용해 그려보자 

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt

# 샘플의 특성 벡터 추출하기

some_digit = X[0]
# print(some_digit)

#  28*28배열로 크기 변환하기

some_digit_image = some_digit.reshape(28,28)
# some_digit_image

In [ ]:
# 변환한 벡터를 맷플롯립 사용해서 그려보기

plt.imshow(some_digit_image, cmap = "binary")
plt.axis("off")
plt.show()

이 이미지는 5처럼 보임.. 실제로도 맞는지 레이블 확인해보자

In [ ]:
print(y[0])
type(y[0])

레이블도 역시 5로 확인 완료...! 하지만 여기서 레이블 자체가 문자열로 되어 있는데 실제 저건 숫자니까

머신러닝 알고리즘에서도 숫자를 기대할것 따라서 y를 정수로 변환해보자

In [ ]:
y = y.astype(np.uint8)
print(y[0])
print(type(y[0]))

자 이제 분류기를 만들어보자

데이터를 자세히 조사하기 위해서는 항상 먼저 테스트 세트를 만들고 따로 떼어놓아야 하지 !!

왜냐하면 미리 테스트해봐야하는데 미리 그것까지 학습하면 안되니까...!

In [ ]:
X_train, X_test, y_train, y_test = X[:60000], X[60000:],y[:60000],y[60000:]

이제 이진 분류기 훈련

우선, 예를 들면 숫자 5만 식별해보자

5-감지기 !

5임, 5 아님을 판단해내는 즉 두 클래스를 구분할 수 있는 이진 분류기를 만들어본느거야

분류 작업을 위한 타깃 벡터 만들기

In [ ]:
# 우선 train과 test모두 5면 True 5가 아니면 False라고 하도록 해야곘지?

y_train_5 = (y_train==5)
y_test_5 = (y_test==5)

확률적 경사 하강법 SGD 분류기로 시작하기

이는 매우 큰 데이터셋을 효율적으로 처리하는 장점을 지님

한번에 하나씩 훈련 샘플을 독립적으로 처리

In [ ]:
# 학습시키기
from sklearn.linear_model import SGDClassifier

sgd_clf = SGDClassifier(random_state=42)
sgd_clf.fit(X_train, y_train_5)

궁금증... train_test 모델에서 왜 항상 random_state 값은 42로 설정할까??



SGDClassifier(random_state=42)

뜨면 학습완료

이번엔 이 모델을 사용해 숫자 5의 이미지를 감지해보자

In [ ]:
sgd_clf.predict([some_digit])

In [ ]:
# 정답 ! 모델이 5임을 맞췄음을 보여준다.

이제 성능을 평가해보는 단계

분류기 평가는 회귀모델보다 후러씬 어렵다...! 

따라서 사용할 수 있는 성능 지표가 많음.

In [ ]:
# 교차 검증을 사용한 정확도 측정

In [ ]:
# from sklearn.model_selection import StratifiedGroupKFold
# from sklearn.base import clone

# skfolds = StratifiedGroupKFold(n_splits=3, random_state=42, shuffle=True)


# for train_index, test_index in skfolds.split(X_train, y_train_5):
#   clone_clf = clone(sgd_clf)
#   X_train_folds = X_train[train_index]
#   y_train_folds = y_train_5[train_index]
#   X_test_fold = X_train[test_index]
#   y_test_fold = y_train_5[test_index]

#   clone_clf.fit(X_train_folds,y_train_folds)
#   y_pred = clone_clf.predict(X_test_fold)
#   n_correct = sum(y_pred == y_test_fold)
  # print(n_correct / len(y_pred))

  # 교차검증 구현과정
  

In [ ]:
from sklearn.model_selection import cross_val_score

cross_val_score(sgd_clf, X_train, y_train_5, cv=3, scoring="accuracy")

In [ ]:
# 더미 분류기를 만들어 비교해보자

from sklearn.base import BaseEstimator

class Never5Classfier(BaseEstimator):
  def fit(self,X,y=None):
    return self
  def predict(self,X):
    return np.zeros((len(X),1),dtype=bool)

In [ ]:
never_5_clf = Never5Classfier()

In [ ]:
cross_val_score(never_5_clf, X_train, y_train_5, cv=3, scoring="accuracy")

오차행렬 조사하는것 - 분류기의 성능을 평가하는 더 좋은 방법

클래스 A의 샘플이 클래스 B로 분류된 회수를 세는 것

In [ ]:
from sklearn.model_selection import cross_val_predict

y_train_pred = cross_val_predict(sgd_clf, X_train, y_train_5, cv=3)

In [ ]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_train_5, y_train_pred)

정밀도와 재현율

F1score . 정밀도와 재현율의 조화평균

F1= 2/((1/정밀도)+(1/재현율))

In [ ]:
from sklearn.metrics import precision_score, recall_score

precision_score(y_train_5, y_train_pred)

In [ ]:
# f1 score 는 f1_score 함수 호출

from sklearn.metrics import f1_score
f1_score(y_train_5, y_train_pred)

정밀도를 올리면 재현율이 줄고 반대도 마찬가지 -> 이를 정밀도/재현율 트레이드 오프라고 함